# **Forecasting U.S. Industrial Production Growth with Robust Regularized Regressions**
*Author:* [Mathijs de Jong](mailto:m.de.jong@tinbergen.nl)

*Date:* 13 October 2020

---
---
### **Introduction**

---
---
### **Methodology**

---
---
### **Preparation**
Before I can start, I import the dependencies, initialize the helper functions, and load and transform the data used for the forecasting experiment.

---
##### **Dependencies**
I start by installing and importing the dependencies used throughout this assignment. Note that some of the packages are installed in a Google Colab environment by default.

In [1]:
'''
Install and import Python dependencies
'''

# Install Python dependencies
!pip install -q dask_kubernetes
!pip install -q fbprophet
!pip install -q joblib
!pip install -q numpy --upgrade
!pip install -q pandas --upgrade
!pip install -q sklearn --upgrade
!pip install -q tqdm --upgrade

# Import Python dependencies
import collections
from dask.distributed import Client
from dask_kubernetes import KubeCluster
from datetime import datetime
from fbprophet import Prophet
from IPython.display import display, HTML
from itertools import product as product
import joblib
import numpy as np
import pandas as pd
import re
from sklearn import linear_model as lm
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.pipeline import Pipeline
from tqdm.notebook import tqdm
import warnings

All results are rounded up to five decimal places in non-scientific notation and warnings are ignored.

In [2]:
'''
Specify notebook hyperparameters
'''

# All results are rounded up to four decimal places
np.set_printoptions(precision = 5, suppress = True)
pd.set_option('display.float_format', lambda x: '{:.5f}'.format(x))

# Ignore warnings
warnings.filterwarnings("ignore")

---
##### **Kubernetes cluster**

In [136]:
# Create Kubernetes cluster on Google Cloud
!gcloud container clusters create \
  --machine-type e2-highcpu-2 \
  --num-nodes 2 \
  --zone europe-west3-a \
  --cluster-version latest \
  tinbergen-cluster 

In [137]:
# Extract credentials into kubeconfig file
!gcloud container clusters get-credentials tinbergen-cluster \
  --zone europe-west3-a \
  --project mphil-tinbergen

Fetching cluster endpoint and auth data.
kubeconfig entry generated for tinbergen-cluster.


In [138]:
# Display available nodes in Kubernetes cluster
!kubectl get node

NAME                                               STATUS   ROLES    AGE     VERSION
gke-tinbergen-cluster-default-pool-bef63592-h2vk   Ready    <none>   7m11s   v1.17.12-gke.1501
gke-tinbergen-cluster-default-pool-bef63592-q5fh   Ready    <none>   7m12s   v1.17.12-gke.1501


In [139]:
# Display available pods in Kubernetes cluster
!kubectl get pods

No resources found in default namespace.


In [140]:
# Add user as admin to cluster
!kubectl create clusterrolebinding cluster-admin-binding \
  --clusterrole=cluster-admin \
  --user=mathijsdejong1995@gmail.com

clusterrolebinding.rbac.authorization.k8s.io/cluster-admin-binding created


In [141]:
# Display the config file
!kubectl config view

apiVersion: v1
clusters:
- cluster:
    certificate-authority-data: DATA+OMITTED
    server: https://35.198.70.185
  name: gke_mphil-tinbergen_europe-west3-a_tinbergen-cluster
contexts:
- context:
    cluster: gke_mphil-tinbergen_europe-west3-a_tinbergen-cluster
    user: gke_mphil-tinbergen_europe-west3-a_tinbergen-cluster
  name: gke_mphil-tinbergen_europe-west3-a_tinbergen-cluster
current-context: gke_mphil-tinbergen_europe-west3-a_tinbergen-cluster
kind: Config
preferences: {}
users:
- name: gke_mphil-tinbergen_europe-west3-a_tinbergen-cluster
  user:
    auth-provider:
      config:
        access-token: ya29.a0AfH6SMD1AubdWKpBNS23NMhWeP9_Pzjqcs-gRjziGQmQdsB4EAsVa4kopkxE5aAZjinPhe258bX5MXvBTJOoTCh6zUz1JpTmNHfyNAew-ZU6ULZgw3cLp7qfvSLu_XpNgF8Iy4i1FSChzdU-V2eAcvKoF_Y3qTtTbWLs5ponj_q_
        cmd-args: config config-helper --format=json
        cmd-path: /Users/mathijs/google-cloud-sdk/bin/gcloud
        expiry: "2020-10-15T15:44:02Z"
        expiry-key: '{.credential.token_expiry}'
 

In [142]:
# Install dask/dask helm chart called tinbergen-helm
!helm repo add dask https://helm.dask.org/
!helm repo update
!helm install tinbergen-helm dask/dask \
  --set scheduler.serviceType=LoadBalancer \
  --set jupyter.serviceType=LoadBalancer \
  -f values.yaml

"dask" already exists with the same configuration, skipping
Hang tight while we grab the latest from your chart repositories...
...Successfully got an update from the "dask" chart repository
...Successfully got an update from the "jupyterhub" chart repository
Update Complete. ⎈Happy Helming!⎈
NAME: tinbergen-helm
LAST DEPLOYED: Thu Oct 15 17:12:09 2020
NAMESPACE: default
STATUS: deployed
REVISION: 1
TEST SUITE: None
NOTES:
Thank you for installing DASK, released at name: tinbergen-helm.

To learn more about the release, try:

  $ helm status tinbergen-helm  # information about running pods and this message
  $ helm get tinbergen-helm     # get full Kubernetes specification

This release includes a Dask scheduler, 5 Dask workers, and 1 Jupyter servers.

The Jupyter notebook server and Dask scheduler expose external services to
which you can connect to manage notebooks, or connect directly to the Dask
cluster. You can get these addresses by running the following:

  export DASK_SCHEDULER

In [149]:
# Display the available kubernetes services
!kubectl get services

NAME                            TYPE           CLUSTER-IP     EXTERNAL-IP      PORT(S)                       AGE
kubernetes                      ClusterIP      10.3.240.1     <none>           443/TCP                       10m
tinbergen-helm-dask-jupyter     LoadBalancer   10.3.244.12    35.234.116.255   80:30273/TCP                  95s
tinbergen-helm-dask-scheduler   LoadBalancer   10.3.253.112   34.107.80.75     8786:30956/TCP,80:32353/TCP   95s


In [152]:
# Initialize Dask Kubernetes cluster
from dask_kubernetes import HelmCluster
cluster = HelmCluster(release_name='tinbergen-helm')

# from dask_kubernetes import KubeCluster
# cluster = KubeCluster.from_yaml('worker-spec.yml')

# Scale cluster
# cluster.scale(10)
# cluster.adapt(minimum=1, maximum=100)

# Display cluster
cluster

ApiException: (404)
Reason: Not Found
HTTP response headers: <CIMultiDictProxy('Audit-Id': '4b6d43c4-918a-4dbe-9f55-4d00c8ab228a', 'Cache-Control': 'no-cache, private', 'Content-Type': 'application/json', 'Date': 'Thu, 15 Oct 2020 15:20:04 GMT', 'Content-Length': '224')>
HTTP response body: {"kind":"Status","apiVersion":"v1","metadata":{},"status":"Failure","message":"services \"tinbergen-helm-scheduler\" not found","reason":"NotFound","details":{"name":"tinbergen-helm-scheduler","kind":"services"},"code":404}



In [151]:
from dask.distributed import Client
client = Client('34.107.80.75:8786')

client

Client Scheduler: tcp://34.107.80.75:8786 Dashboard: http://34.107.80.75:8787/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


In [ ]:
# Create Dask client
from dash.distributed import Client
client = Client(cluster)

# Display dask client
client

In [132]:
# Delete helm chart tinbergen-helm
!helm delete tinbergen-helm

Error: uninstall: Release not loaded: tinbergen-helm: release: not found


In [133]:
# Display Kubernetes namespaces
!kubectl get namespace 

NAME              STATUS   AGE
default           Active   4h15m
kube-node-lease   Active   3h55m
kube-public       Active   4h15m
kube-system       Active   4h15m


In [134]:
# Display Kubernetes clusters on Google Cloud
!gcloud container clusters list

NAME               LOCATION        MASTER_VERSION    MASTER_IP       MACHINE_TYPE  NODE_VERSION      NUM_NODES  STATUS
tinbergen-cluster  europe-west3-a  1.17.12-gke.1501  35.234.121.253  e2-highcpu-2  1.17.12-gke.1501  8          RUNNING


In [135]:
# Delete Kubernetes cluster on Google Cloud
!gcloud container clusters delete tinbergen-cluster --zone=europe-west3-a --quiet

Deleting cluster tinbergen-cluster...done.                                     
Deleted [https://container.googleapis.com/v1/projects/mphil-tinbergen/zones/europe-west3-a/clusters/tinbergen-cluster].


---
##### **Helper functions**
the following helper functions are used for the forecasting experiment.


###### ***Helper function: `transxf`***
Transformation function for the FRED-MD data set based on [McCracken and Ng (2016)](https://doi.org/10.20955/wp.2015.012).

In [3]:
def transxf(x, tcode, tol=1e-6):
  '''
  This function transforms a single series (in a column vector) as specified
  by a given transfromation code. Possible transformations are:
    1) Level: y_{t} = x_{t}
    2) First difference: y_{t} = x_{t} - x_{t-1}
    3) Second difference: y_{t} = (x_{t} - x_{t-1}) - (x_{t-1} - x_{t-2})
    4) Natural log: y_{t} = log(x_{t})
    5) First difference of natural log: y_{t} = log(x_{t}) - log(x_{t-1})
    6) Second difference of natural log:
         y_{t} = (ln(x_{t}) - ln(x_{t-1})) - (ln(x_{t-1}) - ln(x_{t-2}))

  Parameters
  ----------
  x : pandas.Series
    Series to be transformed.
  tcode : float
    Transformation code (1-7).
  tol : float, default=1e-4
    Tolerance for natural logarithm.

  Raises
  ------
  ValueError
    If transformation code is not recognized or the transformation is invalid.

  Returns
  -------
  pandas.Series
    Transformed series.
  '''

  # Apply transformation and update output
  if tcode == 1: return x
  if tcode == 2: return x - x.shift(1)
  if tcode == 3: return x - 2 * x.shift(1) + x.shift(2)
  if tcode == 4 and x.min() > tol: return np.log(x)
  if tcode == 5 and x.min() > tol: return np.log(x) - np.log(x.shift(1))
  if tcode == 6 and x.min() > tol:
    return np.log(x) - 2 * np.log(x.shift(1)) + np.log(x.shift(2))
  if tcode == 7: return x / x.shift(1) - x.shift(1) / x.shift(2)
    
  # Transformation code provided is not implemented
  raise ValueError('Invalid or unknown transformation code: {}'.format(tcode))

###### ***Helper function: `create_exog`***
Function for creating exogenous variables used in the forecasting models.

In [4]:
def create_exog(df, lags, cols=None, hor=1, drop_orig=True, drop_na=True):
  '''
  Create exogenous variables given a dataframe by adding a number of lags to the
  columns of a given dataframe for a specified forecasting horizon.

  Parameters
  ----------
  df : pandas.DataFrame
    Dataframe to add lags to.
  cols : list
    List of columns to add lags for. When None, all columns are lagged. Default
    is None.
  lags : float
    Number of lags to add to the given dataframe.
  hor : float
    Forecast horizon to create exogenous variables for. Default is 1.
  drop_orig :  bool
    Boolean indicating whether to drop the original
    columns that are being lagged. Default is True.
  drop_na :  bool
    Boolean indicating whether to drop rows containing
    np.NaN values.

  Returns
  -------
  pandas.DataFrame
    Dataframe appended with lags.
  '''

  # Determine which columns to add lags to
  if cols is None: cols = df.columns

  # Add lags to the dataframe
  for lag in range(lags):
    df_lags = df[cols].shift(lag + hor)
    df_lags.rename(
      columns = dict(zip(
        cols,
        ['L{}.{}'.format(lag + 1, col) for col in cols]
      )),
      inplace = True
    )
    df = pd.concat([df, df_lags], axis=1)

  # Drop rows containing NaN and/or original series if specified
  if drop_na: df.dropna(inplace=True)
  if drop_orig: df.drop(columns=cols, inplace=True)

  # Return dataframe with lags
  return df

###### ***Helper function: `ar_exog`***
Function for extracting only the lags of the dependent variable out of the exogenous variables.

In [5]:
def ar_exog(X, y):
  '''
  Create exogenous variables given a dataframe by adding a number of lags to the
  columns of a given dataframe for a specified forecasting horizon.

  Parameters
  ----------
  X : pandas.DataFrame or pandas.Series
    Pandas table containing the exogenous variables.
  y : pandas.Series
    Dependent variable to extract the name from.

  Returns
  -------
  pandas.DataFrame
    Dataframe containing only the AR terms of the exogenous variables.
  '''

  # If exegenous variables is pandas.Series, transform it into a dataframe.
  if not isinstance(X, pd.DataFrame): X = pd.DataFrame(X).transpose()
  
  # Extract the AR series in the exogenous data
  ar_cols = []
  for col in X.columns:
    if re.match('^L(\d{1,2})\.' + y.name + '$', col): ar_cols.append(col)

  # Return only AR terms of the exegenous data
  return X[ar_cols]

In [6]:
class TrimmedElasticNet(lm.ElasticNet):
  '''
  Trimmed linear regression with combined L1 and L2 priors as regularizer.

  Parameters
  ----------
  alpha : float, default=1.0
    Constant that multiplies the penalty terms. Defaults to 1.0.
    See the notes for the exact mathematical meaning of this
    parameter. ``alpha = 0`` is equivalent to an ordinary least square,
    solved by the :class:`LinearRegression` object. For numerical
    reasons, using ``alpha = 0`` with the ``Lasso`` object is not advised.
    Given this, you should use the :class:`LinearRegression` object.
  l1_ratio : float, default=0.5
    The ElasticNet mixing parameter, with ``0 <= l1_ratio <= 1``. For
    ``l1_ratio = 0`` the penalty is an L2 penalty. ``For l1_ratio = 1`` it
    is an L1 penalty.  For ``0 < l1_ratio < 1``, the penalty is a
    combination of L1 and L2.
  fit_intercept : bool, default=True
    Whether the intercept should be estimated or not. If ``False``, the
    data is assumed to be already centered.
  normalize : bool, default=False
    This parameter is ignored when ``fit_intercept`` is set to False.
    If True, the regressors X will be normalized before regression by
    subtracting the mean and dividing by the l2-norm.
    If you wish to standardize, please use
    :class:`sklearn.preprocessing.StandardScaler` before calling ``fit``
    on an estimator with ``normalize=False``.
  precompute : bool or array-like of shape (n_features, n_features),\
                default=False
    Whether to use a precomputed Gram matrix to speed up
    calculations. The Gram matrix can also be passed as argument.
    For sparse input this option is always ``True`` to preserve sparsity.
  max_iter : int, default=1000
    The maximum number of iterations
  copy_X : bool, default=True
    If ``True``, X will be copied; else, it may be overwritten.
  tol : float, default=1e-4
    The tolerance for the optimization: if the updates are
    smaller than ``tol``, the optimization code checks the
    dual gap for optimality and continues until it is smaller
    than ``tol``.
  warm_start : bool, default=False
    When set to ``True``, reuse the solution of the previous call to fit as
    initialization, otherwise, just erase the previous solution.
    See :term:`the Glossary <warm_start>`.
  positive : bool, default=False
    When set to ``True``, forces the coefficients to be positive.
  random_state : int, RandomState instance, default=None
    The seed of the pseudo random number generator that selects a random
    feature to update. Used when ``selection`` == 'random'.
    Pass an int for reproducible output across multiple function calls.
    See :term:`Glossary <random_state>`.
  selection : {'cyclic', 'random'}, default='cyclic'
    If set to 'random', a random coefficient is updated every iteration
    rather than looping over features sequentially by default. This
    (setting to 'random') often leads to significantly faster convergence
    especially when tol is higher than 1e-4.
  h : function of the number of targets (n_targets), or int, default=None
    Function or integer for the size of the subset of targets used for trimmed
    least squares.
  init_sets : int, default=500
    Number of initial sets to use in the FAST-LTS algorithm.
  init_iter : int, default=2
    Number of initial C-steps to apply to the initial sets in the FAST-LTS
    algorithm.
  iter_sets : int, default=10
    Number of H sets to use C-steps until convergence for.

  Attributes
  ----------
  coef_ : ndarray of shape (n_features,) or (n_targets, n_features)
      parameter vector (w in the cost function formula)
  sparse_coef_ : sparse matrix of shape (n_features, 1) or \
          (n_targets, n_features)
      ``sparse_coef_`` is a readonly property derived from ``coef_``
  intercept_ : float or ndarray of shape (n_targets,)
      independent term in decision function.
  n_iter_ : list of int
      number of iterations run by the coordinate descent solver to reach
      the specified tolerance.
  '''

  def __init__(self, alpha=1.0, *, l1_ratio=0.5, fit_intercept=True,
              normalize=False, precompute=False, max_iter=1000,
              copy_X=True, tol=1e-4, warm_start=False, positive=False,
              random_state=None, selection='cyclic', h=None, init_sets=500,
              init_iter=2, iter_sets=10):
    self.h = h
    self.init_sets = init_sets
    self.init_iter = init_iter
    self.iter_sets = iter_sets
    self.alpha = alpha
    self.l1_ratio = l1_ratio
    self.fit_intercept = fit_intercept
    self.normalize = normalize
    self.precompute = precompute
    self.max_iter = max_iter
    self.copy_X = copy_X
    self.tol = tol
    self.warm_start = warm_start
    self.positive = positive
    self.random_state = random_state
    self.selection = selection
    

  def __c_step(self, y, X, h_set, iter=10000):
    '''
    Adjusted C-step procedure based Rousseeuw and Van Driessen (2006).
    
    Parameters
    ----------
    y : {pandas.Series}
      Endogenous variable
    X : {pandas.DataFrame}
      Exogenous variables
    h_set : {numpy.array}
      Observations in the H set.
    iter : {int}, default=10000
      Number of iterations of C-step procedure to apply.

    Returns
    -------
    tuple: Updated H set and its associated Q value.
    '''

    for i in range(iter):
      # Estimate model on subset
      super().fit(X.loc[h_set, :], y.loc[h_set])

      # Estimate model and extract squared residuals and SSR on H set
      est = self.predict(X)
      sq_res = np.power(y - est, 2); q = np.sum(sq_res[h_set])

      # Construct new H set
      h_set_new = X.index.values[np.argsort(sq_res)][:self.h]

      # Stop when converged; otherwise, update H set
      if all([idx in h_set_new for idx in h_set]): break
      h_set = h_set_new
    
    # Return the new h set and the SSR for the fitted H set
    return h_set_new, q

  def __subset_size(self, n, p):
    '''
    Set the subset size h for a Least Trimmed Squared model based on the model
    parameters. If h={None}, it is set to the maxmimum of ceil(n * 0.75) and
    ceil((n + p + 1) / 2) in case p <= (n-1). Otherwise, it is set to
    ceil(n * 0.75).

    Parameters
    ----------
    n : {int}
      Total number of observations in the model.
    p : {int}
      Total number of explanatory variables in the model.
    '''

    if callable(self.h): self.h = self.h(n)
    if not self.h:
      if p <= n - 1:
        self.h = int(np.ceil(np.max([(n + p + 1) / 2, 0.75 * n])))
      else:
        self.h = int(np.ceil(0.75 * n))

  def fit(self, X, y, sample_weight=None, check_input=True):
    '''
    Fit linear model using the Fast LTS algorithm by Rousseeuw and Van Driessen
    (2006).
    
    Parameters
    ----------
    X : {array-like, sparse matrix} of shape (n_samples, n_features)
        Training data
    y : array-like of shape (n_samples,) or (n_samples, n_targets)
        Target values. Will be cast to X's dtype if necessary
    sample_weight : array-like of shape (n_samples,), default=None
        Individual weights for each sample

    Returns
    -------
    self : returns an instance of self.
    '''

    # Initialize LTS settings
    n, p = X.shape; h_sets, q_list = [], []; self.__subset_size(n, p)
    
    # If h is equal to the number of observations, return the full fit
    if self.h == n: return super().fit(X, y)
  
    # Construct H set estimates
    for i in range(self.init_sets):
      h_set = np.random.choice(X.index.values, size=3, replace=False)

      # Apply init_iter C-steps to random H set and store results
      h_set, q = self.__c_step(y, X, h_set, iter=self.init_iter)
      h_sets.append(h_set); q_list.append(q)
  
    # Order H sets based on estimation performance
    h_sets = [h_sets[i] for i in np.argsort(q_list)]; q_min = np.inf

    # Carry out C-steps to convergence for iter_sets best H sets
    for i in range(self.iter_sets):
      h_set, q = self.__c_step(y, X, h_sets[i])
      if q < q_min: h_set_opt, q_min = h_set, q 

    # Estimate model using optimal H set
    super().fit(X.loc[h_set_opt, ], y.loc[h_set_opt])

    # Return estimated model based on optimal H set
    return self

In [7]:
class TrimmedLinearRegression(lm.LinearRegression):
  '''
  Ordinary trimmed least squares Linear Regression.

  TrimmedLinearRegression fits a linear model with coefficients
  w = (w1, ..., wp) to minimize the trimmed residual sum of squares between the
  observed targets in the trimmed dataset, and the targets predicted by the
  linear approximation.

  Parameters
  ----------
  fit_intercept : bool, default=True
    Whether to calculate the intercept for this model. If set
    to False, no intercept will be used in calculations
    (i.e. data is expected to be centered).
  normalize : bool, default=False
    This parameter is ignored when ``fit_intercept`` is set to False.
    If True, the regressors X will be normalized before regression by
    subtracting the mean and dividing by the l2-norm.
    If you wish to standardize, please use
    :class:`sklearn.preprocessing.StandardScaler` before calling ``fit`` on
    an estimator with ``normalize=False``.
  copy_X : bool, default=True
    If True, X will be copied; else, it may be overwritten.
  n_jobs : int, default=None
    The number of jobs to use for the computation. This will only provide
    speedup for n_targets > 1 and sufficient large problems.
    ``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.
    ``-1`` means using all processors. See :term:`Glossary <n_jobs>`
    for more details.
  h : function of the number of targets (n_targets), or int, default=None
    Function or integer for the size of the subset of targets used for trimmed
    least squares.
  init_sets : int, default=500
    Number of initial sets to use in the FAST-LTS algorithm.
  init_iter : int, default=2
    Number of initial C-steps to apply to the initial sets in the FAST-LTS
    algorithm.
  iter_sets : int, default=10
    Number of H sets to use C-steps until convergence for.

  Attributes
  ----------
  coef_ : array of shape (n_features, ) or (n_targets, n_features)
      Estimated coefficients for the linear regression problem.
      If multiple targets are passed during the fit (y 2D), this
      is a 2D array of shape (n_targets, n_features), while if only
      one target is passed, this is a 1D array of length n_features.
  rank_ : int
      Rank of matrix `X`. Only available when `X` is dense.
  singular_ : array of shape (min(X, y),)
      Singular values of `X`. Only available when `X` is dense.
  intercept_ : float or array of shape (n_targets,)
      Independent term in the linear model. Set to 0.0 if
      `fit_intercept = False`.
  '''

  def __init__(self, *, fit_intercept=True, normalize=False, copy_X=True, 
               n_jobs=None, h=None, init_sets=500, init_iter=2, iter_sets=10):
    self.h = h
    self.init_sets = init_sets
    self.init_iter = init_iter
    self.iter_sets = iter_sets
    self.fit_intercept = fit_intercept
    self.normalize = normalize
    self.copy_X = copy_X
    self.n_jobs = n_jobs

  def __c_step(self, y, X, h_set, iter=10000):
    '''
    Adjusted C-step procedure based Rousseeuw and Van Driessen (2006).
    
    Parameters
    ----------
    y : {pandas.Series}
      Endogenous variable
    X : {pandas.DataFrame}
      Exogenous variables
    h_set : {numpy.array}
      Observations in the H set.
    iter : {int}, default=10000
      Number of iterations of C-step procedure to apply.

    Returns
    -------
    tuple: Updated H set and its associated Q value.
    '''

    for i in range(iter):
      # Estimate model on subset
      super().fit(X.loc[h_set, :], y.loc[h_set])

      # Estimate model and extract squared residuals and SSR on H set
      est = self.predict(X)
      sq_res = np.power(y - est, 2); q = np.sum(sq_res[h_set])

      # Construct new H set
      h_set_new = X.index.values[np.argsort(sq_res)][:self.h]

      # Stop when converged; otherwise, update H set
      if all([idx in h_set_new for idx in h_set]): break
      h_set = h_set_new
    
    # Return the new h set and the SSR for the fitted H set
    return h_set_new, q

  def __subset_size(self, n, p):
    '''
    Set the subset size h for a Least Trimmed Squared model based on the model
    parameters. If h={None}, it is set to the maxmimum of ceil(n * 0.75) and
    ceil((n + p + 1) / 2) in case p <= (n-1). Otherwise, it is set to
    ceil(n * 0.75).

    Parameters
    ----------
    n : {int}
      Total number of observations in the model.
    p : {int}
      Total number of explanatory variables in the model.
    '''

    if callable(self.h): self.h = self.h(n)
    if not self.h:
      if p <= n - 1:
        self.h = int(np.ceil(np.max([(n + p + 1) / 2, 0.75 * n])))
      else:
        self.h = int(np.ceil(0.75 * n))

  def fit(self, X, y, sample_weight=None, check_input=True):
    '''
    Fit linear model using the Fast LTS algorithm by Rousseeuw and Van Driessen
    (2006).
    
    Parameters
    ----------
    X : {array-like, sparse matrix} of shape (n_samples, n_features)
        Training data
    y : array-like of shape (n_samples,) or (n_samples, n_targets)
        Target values. Will be cast to X's dtype if necessary
    sample_weight : array-like of shape (n_samples,), default=None
        Individual weights for each sample

    Returns
    -------
    self : returns an instance of self.
    '''

    # Initialize LTS settings
    n, p = X.shape; h_sets, q_list = [], []; self.__subset_size(n, p)

    # If h is equal to the number of observations, return the full fit
    if self.h == n: return super().fit(X, y)
  
    # Construct H set estimates
    for i in range(self.init_sets):
      h_set = np.random.choice(X.shape.values, size=3, replace=False)

      # Apply init_iter C-steps to random H set and store results
      h_set, q = self.__c_step(y, X, h_set, iter=self.init_iter)
      h_sets.append(h_set); q_list.append(q)
  
    # Order H sets based on estimation performance
    h_sets = [h_sets[i] for i in np.argsort(q_list)]; q_min = np.inf

    # Carry out C-steps to convergence for iter_sets best H sets
    for i in range(self.iter_sets):
      h_set, q = self.__c_step(y, X, h_sets[i])
      if q < q_min: h_set_opt, q_min = h_set, q 

    # Estimate model using optimal H set
    super().fit(X.loc[h_set_opt, ], y.loc[h_set_opt])

    # Return estimated model based on optimal H set
    return self

###### ***Helper function: `prophet_mod`***
Implementation of the [Prophet time series model](https://facebook.github.io/prophet/) that can be used by the `fast_lts` function defined above.

In [8]:
def prophet_mod(X, y):
  '''
  Function for a Prophet model.

  Parameters
  ----------
  X : pandas.DataFrame
    Dataframe containing the exogenous variables.
  y : pandas.Series
    Series of the endogenous variable.
  weights : list of float
    Not supported by the scikit-learn package.

  Returns
  -------
  Prophet
    Fitted Prophet model.
  '''

  # Create dataframe in the right format
  df = X.copy(); df['y'] = y; df.reset_index(inplace=True)
  df.rename(columns={'index' : 'ds'}, inplace=True)
  
  # Create Prophet model
  mod = Prophet(daily_seasonality=False, weekly_seasonality=False)
  for col in X.columns:
    if y.name not in col: mod.add_regressor(col, mode='additive')
  
  # Fit Prophet model
  mod.fit(df)
  
  # Return fited Prophet model
  return mod

---
##### **Data**
The data used for this experiment is the FRED-MD data set from the personal website of Michael McCracken. The following code is used to download the code directly into this Google Colab instance.

In [9]:
'''
Load data into cloud instance
'''

# Import data from FRED-MD
!wget -nc -q -O 'current.csv' https://s3.amazonaws.com/files.fred.stlouisfed.org/fred-md/monthly/current.csv

Next, the data is transformed using the transformations described in [McCracken and Ng (2016)](https://doi.org/10.20955/wp.2015.012).

In [10]:
'''
Load and transform data in notebook
'''

# Load data into pandas.DataFrame
df_fred = pd.read_csv('current.csv', index_col=0)

# Drop name index column
df_fred.index.name = None

# Transform data
df = df_fred.drop(labels=['Transform:', np.NaN], axis=0)
for col_name in df.columns:
  df[col_name] = transxf(df[col_name], df_fred.at['Transform:', col_name])

# Replace NaN values by previous observation and drop first row
df.fillna(method='ffill', inplace=True)
drop_rows = ['1/1/1959', '2/1/1959']
df.drop(labels=drop_rows, inplace=True)

# Drop columns that still contain NaN values
df.drop(columns=df.columns[df.isna().sum() > 0], inplace=True)

# Display which periods and series have been dropped
print('The following time periods have been dropped: {}'.format(
  ', '.join(list(
    set(df_fred.index.dropna()) - set(df.index) - set(['Transform:'])
  ))
))
print('The following series have been dropped: {}'.format(
  ', '.join(list(set(df_fred.columns) - set(df.columns)))
))

# Change index format to pandas.Timestamp
df.index = pd.Series(df.index.values).apply(
  lambda date_str: pd.Timestamp(datetime.strptime(date_str, '%m/%d/%Y'))
)

The following time periods have been dropped: 1/1/1959, 2/1/1959
The following series have been dropped: PERMITNE, PERMITMW, UMCSENTx, ACOGNO, ANDENOx, TWEXAFEGSMTHx, PERMIT, PERMITS, VXOCLSx, PERMITW


A sanity check is put in place to ensure that the data is correctly loaded into this notebook.

In [11]:
'''
Sanity checks for loading and transforming data
'''

# Display head of raw data
display(HTML(df_fred.head().to_html()))

# Display head of transformed data
display(HTML(df.head().to_html()))

# Describe transformed data
display(HTML(df.describe().to_html()))

,RPI,W875RX1,DPCERA3M086SBEA,CMRMTSPLx,RETAILx,INDPRO,IPFPNSS,IPFINAL,IPCONGD,IPDCONGD,IPNCONGD,IPBUSEQ,IPMAT,IPDMAT,IPNMAT,IPMANSICS,IPB51222S,IPFUELS,CUMFNS,HWI,HWIURATIO,CLF16OV,CE16OV,UNRATE,UEMPMEAN,UEMPLT5,UEMP5TO14,UEMP15OV,UEMP15T26,UEMP27OV,CLAIMSx,PAYEMS,USGOOD,CES1021000001,USCONS,MANEMP,DMANEMP,NDMANEMP,SRVPRD,USTPU,USWTRADE,USTRADE,USFIRE,USGOVT,CES0600000007,AWOTMAN,AWHMAN,HOUST,HOUSTNE,HOUSTMW,HOUSTS,HOUSTW,PERMIT,PERMITNE,PERMITMW,PERMITS,PERMITW,ACOGNO,AMDMNOx,ANDENOx,AMDMUOx,BUSINVx,ISRATIOx,M1SL,M2SL,M2REAL,BOGMBASE,TOTRESNS,NONBORRES,BUSLOANS,REALLN,NONREVSL,CONSPI,S&P 500,S&P: indust,S&P div yield,S&P PE ratio,FEDFUNDS,CP3Mx,TB3MS,TB6MS,GS1,GS5,GS10,AAA,BAA,COMPAPFFx,TB3SMFFM,TB6SMFFM,T1YFFM,T5YFFM,T10YFFM,AAAFFM,BAAFFM,TWEXAFEGSMTHx,EXSZUSx,EXJPUSx,EXUSUKx,EXCAUSx,WPSFD49207,WPSFD49502,WPSID61,WPSID62,OILPRICEx,PPICMM,CPIAUCSL,CPIAPPSL,CPITRNSL,CPIMEDSL,CUSR0000SAC,CUSR0000SAD,CUSR0000SAS,CPIULFSL,CUSR0000SA0L2,CUSR0000SA0L5,PCEPI,DDURRG3M086SBEA,DNDGRG3M086SBEA,DSERRG3M086SBEA,CES0600000008,CES2000000008,CES3000000008,UMCSENTx,MZMSL,DTCOLNVHFNM,DTCTHFNM,INVEST,VXOCLSx
Transform:,5.00000,5.00000,5.00000,5.00000,5.00000,5.00000,5.00000,5.00000,5.00000,5.00000,5.00000,5.00000,5.00000,5.00000,5.00000,5.00000,5.00000,5.00000,2.00000,2.00000,2.00000,5.00000,5.00000,2.00000,2.00000,5.00000,5.00000,5.00000,5.00000,5.00000,5.00000,5.00000,5.00000,5.00000,5.00000,5.00000,5.00000,5.00000,5.00000,5.00000,5.00000,5.00000,5.00000,5.00000,1.00000,2.00000,1.00000,4.00000,4.00000,4.00000,4.00000,4.00000,4.00000,4.00000,4.00000,4.00000,4.00000,5.00000,5.00000,5.00000,5.00000,5.00000,2.00000,6.00000,6.00000,5.00000,6.00000,6.00000,7.00000,6.00000,6.00000,6.00000,2.00000,5.00000,5.00000,2.00000,5.00000,2.00000,2.00000,2.00000,2.00000,2.00000,2.00000,2.00000,2.00000,2.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,5.00000,5.00000,5.00000,5.00000,5.00000,6.00000,6.00000,6.00000,6.00000,6.00000,6.00000,6.00000,6.00000,6.00000,6.00000,6.00000,6.00000,6.00000,6.00000,6.00000,6.00000,6.00000,6.00000,6.00000,6.00000,6.00000,6.00000,6.00000,2.00000,6.00000,6.00000,6.00000,6.00000,1.00000
1/1/1959,2437.29600,2288.80000,17.30200,292258.83290,18235.77392,22.62500,23.45810,22.19040,32.40780,21.98820,37.72800,7.99550,21.21460,12.60470,30.53720,21.14920,20.25900,38.34820,80.19730,1357.00000,0.33358,67936.00000,63868.00000,6.00000,16.30000,1574.00000,1169.00000,1396.00000,594.00000,802.00000,291078.00000,52478.00000,18796.00000,713.00000,2993.00000,14998.00000,8740.00000,6258.00000,33682.00000,10774.00000,2568.70000,5350.30000,2418.00000,8105.00000,39.80000,2.50000,40.20000,1657.00000,350.00000,452.00000,505.00000,350.00000,nan,nan,nan,nan,nan,nan,14716.48296,nan,42620.34624,84889.55898,1.56000,138.90000,286.60000,987.90000,50463.00000,18.90000,18338.00000,35.21300,24.92420,48.96116,0.12496,55.62000,59.30000,3.15834,18.44574,2.48000,3.30000,2.82000,3.09000,3.36000,4.01000,4.02000,4.12000,4.87000,0.82000,0.34000,0.61000,0.88000,1.53000,1.54000,1.64000,2.39000,nan,4.31220,359.84170,2.80650,0.96710,33.10000,33.40000,30.60000,31.60000,3.00000,32.50000,29.01000,44.80000,29.30000,21.10000,33.30000,38.10000,22.90000,28.90000,30.70000,29.60000,16.07400,56.91800,17.79100,11.35800,2.13000,2.45000,2.04000,nan,274.90000,6476.00000,12298.00000,84.20430,nan
2/1/1959,2446.90200,2297.00000,17.48200,294429.54530,18369.56308,23.06810,23.77470,22.38270,32.64550,22.10360,38.08860,8.10250,21.88640,13.18530,31.07190,21.53790,20.20380,37.82580,81.44280,1421.00000,0.35839,67649.00000,63684.00000,5.90000,15.50000,1554.00000,1164.00000,1277.00000,545.00000,732.00000,282958.00000,52688.00000,18890.00000,704.20000,2980.00000,15115.00000,8839.00000,6276.00000,33798.00000,10816.00000,2575.40000,5381.30000,2420.00000,8116.00000,39.70000,2.60000,40.30000,1667.00000,346.00000,469.00000,508.00000,344.00000,nan,nan,nan,nan,nan,nan,15400.24873,nan,43677.15151,85181.81131,1.53934,139.40000,287.70000,992.10000,49805.00000,18.60000,18065.00000,35.22010,2

,RPI,W875RX1,DPCERA3M086SBEA,CMRMTSPLx,RETAILx,INDPRO,IPFPNSS,IPFINAL,IPCONGD,IPDCONGD,IPNCONGD,IPBUSEQ,IPMAT,IPDMAT,IPNMAT,IPMANSICS,IPB51222S,IPFUELS,CUMFNS,HWI,HWIURATIO,CLF16OV,CE16OV,UNRATE,UEMPMEAN,UEMPLT5,UEMP5TO14,UEMP15OV,UEMP15T26,UEMP27OV,CLAIMSx,PAYEMS,USGOOD,CES1021000001,USCONS,MANEMP,DMANEMP,NDMANEMP,SRVPRD,USTPU,USWTRADE,USTRADE,USFIRE,USGOVT,CES0600000007,AWOTMAN,AWHMAN,HOUST,HOUSTNE,HOUSTMW,HOUSTS,HOUSTW,AMDMNOx,AMDMUOx,BUSINVx,ISRATIOx,M1SL,M2SL,M2REAL,BOGMBASE,TOTRESNS,NONBORRES,BUSLOANS,REALLN,NONREVSL,CONSPI,S&P 500,S&P: indust,S&P div yield,S&P PE ratio,FEDFUNDS,CP3Mx,TB3MS,TB6MS,GS1,GS5,GS10,AAA,BAA,COMPAPFFx,TB3SMFFM,TB6SMFFM,T1YFFM,T5YFFM,T10YFFM,AAAFFM,BAAFFM,EXSZUSx,EXJPUSx,EXUSUKx,EXCAUSx,WPSFD49207,WPSFD49502,WPSID61,WPSID62,OILPRICEx,PPICMM,CPIAUCSL,CPIAPPSL,CPITRNSL,CPIMEDSL,CUSR0000SAC,CUSR0000SAD,CUSR0000SAS,CPIULFSL,CUSR0000SA0L2,CUSR0000SA0L5,PCEPI,DDURRG3M086SBEA,DNDGRG3M086SBEA,DSERRG3M086SBEA,CES0600000008,CES2000000008,CES3000000008,MZMSL,DTCOLNVHFNM,DTCTHFNM,INVEST
1959-03-01,0.00643,0.00737,0.00939,-0.00342,0.00832,0.01430,0.00603,0.00489,0.00000,0.01940,-0.00474,0.01074,0.02564,0.03864,0.01491,0.01553,0.00680,0.02500,1.03410,103.00000,0.04256,0.00617,0.00911,-0.30000,-0.20000,-0.06308,-0.06294,-0.05389,-0.02791,-0.07369,-0.08329,0.00617,0.00943,-0.00014,0.01101,0.00948,0.01415,0.00286,0.00434,0.00526,0.00349,0.00930,0.00412,0.00197,40.00000,0.20000,40.40000,7.39018,5.79909,6.02345,6.22059,5.92426,0.02217,0.02497,0.00513,-0.01033,-0.00144,0.00137,0.00623,0.01168,0.00519,0.00199,-0.00275,-0.00438,-0.00129,0.00036,0.02506,0.02472,-0.06781,0.03107,0.37000,0.09000,0.10000,0.00000,0.07000,0.03000,0.03000,-0.01000,-0.04000,0.55000,0.00000,0.33000,0.81000,1.19000,1.19000,1.33000,2.05000,0.00220,0.00000,0.00121,-0.00514,-0.00302,-0.00300,-0.00326,0.00953,-0.01005,0.01223,-0.00069,0.00223,0.00337,-0.00002,-0.00301,0.00524,-0.00436,0.00000,0.00000,0.00000,-0.00025,0.00067,-0.00112,0.00026,-0.00002,-0.00815,0.00482,0.00107,0.00493,0.00414,-0.01479
1959-04-01,0.00650,0.00702,-0.00358,0.01993,0.00062,0.02108,0.01434,0.01455,0.01566,0.00638,0.01651,0.02579,0.02725,0.02965,0.03050,0.02110,0.00405,-0.00450,1.51530,65.00000,0.04403,0.00397,0.00777,-0.40000,-0.40000,0.02371,-0.15721,-0.15236,-0.26161,-0.07479,-0.05500,0.00577,0.01043,0.00156,0.02362,0.00822,0.01242,0.00222,0.00315,0.00294,0.00483,-0.00669,0.00370,0.00123,40.20000,0.10000,40.50000,7.37149,5.61677,5.96871,6.28413,5.96101,0.01099,0.01641,0.01334,0.00000,-0.00215,-0.00209,0.00270,0.00796,0.02698,0.02153,0.01465,0.00421,-0.00086,0.00007,0.01660,0.01872,-0.04020,0.01430,0.16000,0.07000,0.15000,0.14000,0.11000,0.13000,0.13000,0.10000,0.01000,0.46000,-0.01000,0.31000,0.76000,1.16000,1.16000,1.27000,1.90000,-0.00005,0.00000,0.00135,-0.00641,0.00000,0.00600,0.00000,0.00315,0.01005,-0.01833,0.00138,0.00223,-0.00341,-0.00471,0.00301,-0.00524,0.00434,0.00345,0.00000,0.00000,0.00130,-0.00023,0.00135,0.00183,-0.00002,0.01220,-0.00489,-0.00254,0.01213,0.00673,0.02493
1959-05-01,0.00583,0.00663,0.01198,0.00680,0.00780,0.01495,0.00827,0.00958,0.00477,0.02015,0.00000,0.03190,0.02543,0.03405,0.00894,0.01382,0.01474,-0.01935,0.92370,66.00000,0.03074,-0.00236,-0.00107,-0.10000,-0.20000,-0.01009,0.07327,-0.07389,-0.04512,-0.09294,0.00218,0.00429,0.00564,0.00678,0.00065,0.00661,0.00910,0.00301,0.00352,0.00403,0.00404,0.01122,0.00327,0.00135,40.30000,0.00000,40.70000,7.31189,5.56068,5.87212,6.18826,5.98141,-0.04304,-0.00304,0.00571,-0.01033,0.00713,0.00411,0.00518,-0.00544,-0.02153,-0.01837,0.00872,-0.00059,0.00165,0.00057,0.01495,0.01902,-0.03468,0.01141,-0.06000,0.14000,-0.11000,0.06000,0.24000,0.23000,0.19000,0.14000,0.10000,0.66000,-0.06000,0.43000,1.06000,1.45000,1.41000,1.47000,2.06000,0.00005,0.00000,-0.00071,-0.00062,0.00301,-0.00600,0.00649,-0.01266,0.00000,0.01220,0.00172,-0.00000,-0.00337,0.00468,0.00301,0.00261,-0.00002,-0.00001,0.00000,0.00000,-0.00149,0.00065,-0.00169,-0.00202,-0.00002,-0.00409,-0.00482,0.00465,0.00283,0.00202,-0.01534
1959-06-01,

,RPI,W875RX1,DPCERA3M086SBEA,CMRMTSPLx,RETAILx,INDPRO,IPFPNSS,IPFINAL,IPCONGD,IPDCONGD,IPNCONGD,IPBUSEQ,IPMAT,IPDMAT,IPNMAT,IPMANSICS,IPB51222S,IPFUELS,CUMFNS,HWI,HWIURATIO,CLF16OV,CE16OV,UNRATE,UEMPMEAN,UEMPLT5,UEMP5TO14,UEMP15OV,UEMP15T26,UEMP27OV,CLAIMSx,PAYEMS,USGOOD,CES1021000001,USCONS,MANEMP,DMANEMP,NDMANEMP,SRVPRD,USTPU,USWTRADE,USTRADE,USFIRE,USGOVT,CES0600000007,AWOTMAN,AWHMAN,HOUST,HOUSTNE,HOUSTMW,HOUSTS,HOUSTW,AMDMNOx,AMDMUOx,BUSINVx,ISRATIOx,M1SL,M2SL,M2REAL,BOGMBASE,TOTRESNS,NONBORRES,BUSLOANS,REALLN,NONREVSL,CONSPI,S&P 500,S&P: indust,S&P div yield,S&P PE ratio,FEDFUNDS,CP3Mx,TB3MS,TB6MS,GS1,GS5,GS10,AAA,BAA,COMPAPFFx,TB3SMFFM,TB6SMFFM,T1YFFM,T5YFFM,T10YFFM,AAAFFM,BAAFFM,EXSZUSx,EXJPUSx,EXUSUKx,EXCAUSx,WPSFD49207,WPSFD49502,WPSID61,WPSID62,OILPRICEx,PPICMM,CPIAUCSL,CPIAPPSL,CPITRNSL,CPIMEDSL,CUSR0000SAC,CUSR0000SAD,CUSR0000SAS,CPIULFSL,CUSR0000SA0L2,CUSR0000SA0L5,PCEPI,DDURRG3M086SBEA,DNDGRG3M086SBEA,DSERRG3M086SBEA,CES0600000008,CES2000000008,CES3000000008,MZMSL,DTCOLNVHFNM,DTCTHFNM,INVEST
count,738.00000,738.00000,738.00000,738.00000,738.00000,738.00000,738.00000,738.00000,738.00000,738.00000,738.00000,738.00000,738.00000,738.00000,738.00000,738.00000,738.00000,738.00000,738.00000,738.00000,738.00000,738.00000,738.00000,738.00000,738.00000,738.00000,738.00000,738.00000,738.00000,738.00000,738.00000,738.00000,738.00000,738.00000,738.00000,738.00000,738.00000,738.00000,738.00000,738.00000,738.00000,738.00000,738.00000,738.00000,738.00000,738.00000,738.00000,738.00000,738.00000,738.00000,738.00000,738.00000,738.00000,738.00000,738.00000,738.00000,738.00000,738.00000,738.00000,738.00000,738.00000,738.00000,738.00000,738.00000,738.00000,738.00000,738.00000,738.00000,738.00000,738.00000,738.00000,738.00000,738.00000,738.00000,738.00000,738.00000,738.00000,738.00000,738.00000,738.00000,738.00000,738.00000,738.00000,738.00000,738.00000,738.00000,738.00000,738.00000,738.00000,738.00000,738.00000,738.00000,738.00000,738.00000,738.00000,738.00000,738.00000,738.00000,738.00000,738.00000,738.00000,738.00000,738.00000,738.00000,738.00000,738.00000,738.00000,738.00000,738.00000,738.00000,738.00000,738.00000,738.00000,738.00000,738.00000,738.00000,738.00000,738.00000
mean,0.00271,0.00242,0.00259,0.00240,0.00457,0.00201,0.00193,0.00200,0.00156,0.00229,0.00130,0.00327,0.00211,0.00271,0.00155,0.00205,0.00238,0.00134,-0.01517,7.87805,0.00015,0.00117,0.00114,0.00339,0.00637,0.00052,0.00134,0.00251,0.00336,0.00108,0.00175,0.00133,0.00007,-0.00030,0.00120,-0.00030,-0.00021,-0.00043,0.00173,0.00121,0.00105,0.00139,0.00173,0.00135,40.27466,0.00149,40.76341,7.22056,5.06789,5.55495,6.40567,5.77972,0.00368,0.00434,0.00422,-0.00034,0.00001,0.00000,0.00266,0.00005,0.00006,0.00007,-0.00002,-0.00002,-0.00001,0.00004,0.00559,0.00598,-0.00208,0.00052,-0.00316,-0.00327,-0.00352,-0.00408,-0.00462,-0.00500,-0.00449,-0.00256,-0.00220,0.11661,-0.44736,-0.30843,0.05801,0.72562,1.04652,2.04175,3.04913,-0.00211,-0.00166,-0.00103,0.00041,-0.00000,-0.00000,0.00000,0.00010,0.00005,0.00002,0.00001,0.00001,0.00001,-0.00001,0.00001,0.00002,-0.00000,0.00001,0.00001,0.00001,0.00000,0.00000,-0.00001,0.00000,-0.00000,0.00000,-0.00001,-0.00000,0.00001,0.00002,0.00003
std,0.00726,0.00621,0.00839,0.01288,0.01516,0.00994,0.01012,0.01104,0.01103,0.03409,0.00789,0.01667,0.01204,0.01890,0.01118,0.01157,0.03527,0.02091,0.84196,180.97717,0.04288,0.00314,0.00679,0.43737,0.80649,0.09381,0.08707,0.06361,0.09200,0.07106,0.11329,0.00612,0.00661,0.01831,0.01077,0.00621,0.00783,0.00458,0.00615,0.00545,0.00353,0.00709,0.00225,0.00345,0.65037,0.14280,0.74173,0.30915,0.41746,0.41793,0.29922,0.37478,0.03960,0.01054,0.00590,0.02099,0.00968,0.00384,0.00569,0.02105,0.06395,1.04825,0.00856,0.00477,0.00764,0.00126,0.03603,0.03614,0.11935,0.04567,0.50977,0.49180,0.42106,0.39676,0.41346,0.31806,0.27479,0.21838,0.21200,0.43532,0.70349,0.75908,0.76407,1.33757,1.61017,1.92196,2.02501,0.02475,0.02338,0.02147,0.01337,0.00704,0.00911,0.00751,0.04121,0.10293,0.03168,0.00270,0.00585,0.01128,0.00

---
---
### **Forecasting experiment**

---
#### **Setup**
In this forecasting experiment, I investigate the forecasting performance for forecasting macroeconomic data of different regression models. The variable of interest in this experiment is the U.S. Industrial Production.

In [12]:
'''
Construct the endogenous variable
'''

# Specify dependent variable
dep_var = 'INDPRO'

# Construct the endogenous variable
endog = df[dep_var]

The full FRED-MD data as described in [McCracken and Ng (2016)](https://doi.org/10.20955/wp.2015.012) is used for this experiment. The in-sample period is until the end of 2007 and the out-of-sample period starts from 2008 until the latest month currently available. The end-of-sample month is set so that the results can be easily reproduced.

In [13]:
'''
Specify the in-sample and out-of-sample windows for forecasting
'''

# End of in-sample period
end_is = '12/1/2007'

# End of out-of-sample period
end_oos = '9/1/2020'

# Construct list of out-of-sample months
oos_months = df.index[(df.index > end_is) & (df.index <= end_oos)]

I compare several methods using all the data available in the data set. The forecasting horizons that I am considering are $h = 1$ and $h = 12$. The different lags of variables included are $p = 1$, $p = 12$ and $p=24$.

The different regression methods used for this experiment are standard OLS, Ridge regression, standard Least Trimmed Squares (OLTS), that is, the standard OLS model estimated through Least Trimmed Squares (LTS) and a Trimmed Ridge regression (TRidge). All models are compared to the perpetual mean (PM) and $AR(1)$ benchmark models. For the LTS methods, I use 95% of the data and hence leave out 5% for robustness, that is, $h = \lceil 0.95 \times n\rceil$.

In [14]:
'''
Specify forecasting experiment settings
'''

# Set the horizons and lags
hors = {'h=1' : 1, 'h=12' : 12}
lags = {'p=1' : 1, 'p=12' : 12, 'p=24' : 24}

# Initialize dictionaries for forecast errors and results
fe = collections.defaultdict(lambda : collections.defaultdict(dict))
res_dict = collections.defaultdict(dict)

# Set the number of observations in LTS to ceil(0.95 * n)
h = lambda n: int(np.ceil(0.95 * n))

# Initialize model specific hyperparameters
init_sets = 100
n_alphas = 10
n_ratios = 10
cvts_splits = 3

# Initialize the hyperparameters for grid search
l1_ratios = np.linspace(start=0, stop=1, num=n_ratios)
alphas = lm.ElasticNetCV(n_alphas=n_alphas).fit(
  X = df.loc[:end_is, df.columns[df.columns != dep_var]],
  y = df.loc[:end_is, dep_var]
).alphas_
elast_params = {'alpha' : alphas, 'l1_ratio' : l1_ratios}

In [124]:
'''
Initialize the dictionary with forecasting methods
'''

methods = {
  'PM'   : lambda y, X: lm.LinearRegression(normalize=False).fit(
                          X = np.ones((X.shape[0], 1)),
                          y = y
                        ),
  # 'PAR'  : lambda y, X: prophet_mod(
  #                         X = ar_exog(X, y),
  #                         y = y
  #                       ),
  # 'PARX'  : lambda y, X: prophet_mod(
  #                         X = X,
  #                         y = y
  #                       ),
  # 'AR'   : lambda y, X: lm.LinearRegression(normalize=True).fit(
  #                         X = ar_exog(X, y),
  #                         y = y
  #                       ),
  # 'ARX'  : lambda y, X: lm.LinearRegression(normalize=True).fit(
  #                         X = X,
  #                         y = y
  #                       ),
  'EAR'  : lambda y, X: GridSearchCV(
                          estimator = lm.ElasticNet(normalize=True),
                          param_grid = elast_params,
                          scoring = 'r2',
                          cv = TimeSeriesSplit(n_splits=cvts_splits),
                          n_jobs = -1
                        ).fit(
                          X = ar_exog(X, y),
                          y = y
                        ),
  'EARX' : lambda y, X: GridSearchCV(
                          estimator = lm.ElasticNet(normalize=True),
                          param_grid = elast_params,
                          scoring = 'r2',
                          cv = TimeSeriesSplit(n_splits=cvts_splits),
                          n_jobs = -1
                        ).fit(
                          X = X,
                          y = y
                        ),
  # 'TAR'  : lambda y, X: TrimmedLinearRegression(
  #                         normalize = True,
  #                         h = h,
  #                         init_sets = init_sets
  #                       ).fit(
  #                         X = ar_exog(X, y),
  #                         y = y
  #                       ),
  # 'TARX' : lambda y, X: TrimmedLinearRegression(
  #                         normalize = True,
  #                         h = h,
  #                         init_sets = init_sets
  #                       ).fit(
  #                         X = X,
  #                         y = y
  #                       ),
  'TEAR' : lambda y, X: GridSearchCV(
                          estimator = TrimmedElasticNet(
                            normalize = True,
                            h = h,
                            init_sets = init_sets
                          ),
                          param_grid = elast_params,
                          scoring = 'r2',
                          cv = TimeSeriesSplit(n_splits=cvts_splits),
                          n_jobs = -1
                        ).fit(
                          X = ar_exog(X, y),
                          y = y
                        ),
  # 'TEARX': lambda y, X: GridSearchCV(
  #                         estimator = TrimmedElasticNet(
  #                           normalize = True,
  #                           h = h,
  #                           init_sets = init_sets
  #                         ),
  #                         param_grid = elast_params,
  #                         scoring = 'r2',
  #                         cv = TimeSeriesSplit(n_splits=cvts_splits),
  #                         n_jobs = -1
  #                       ).fit(
  #                         X = X, 
  #                         y = y
  #                       )
}

---
#### **Experiment**
The following code is used to construct the forecast errors.

In [125]:
'''
Generate forecasting results
'''

# Initialize progress bar for horizons and lags
pb = tqdm(list(product(hors.keys(), lags.keys())), leave=False)

# For all horizons and lags, estimate the forecasting errors
for hor, lag in pb:
  # Update progress bar message
  pb.set_description('Construct forecast for {} and {}'.format(hor, lag))

  # Construct variables for in-sample window
  X = create_exog(df, lags[lag], hor=hors[hor]); y = endog[X.index]

  # Initialize progress bar for methods
  pb_method = tqdm(methods.keys(), leave=False)
  for method in methods: fe[hor][lag][method] = np.array([])

  # For horizon and lag, for all methods estimate forecasting errors
  for method in pb_method:
    pb_method.set_description('Method: {}'.format(method))
    
    # Initilize progress bar for months
    pb_month = tqdm(oos_months, leave=False)

    # For horizon, lag and method, for each month estimate forecasting error
    for oos_m in pb_month:

      # Only estimate Prophet models for feasible periods
      if ('PAR' in method) and (hors[hor] > 1): continue

      # Update progress bar message
      pb_month.set_description('Month: {}'.format(oos_m.strftime('%m-%Y')))

      # Estimate forecasting model
      with joblib.parallel_backend('dask'):
        est_mod = methods[method](y[y.index < oos_m], X.loc[X.index < oos_m, :])

      # Construct forecast
      if method == 'PM':
        est = est_mod.coef_[0]
      elif 'PAR' in method:
        X_pred = pd.DataFrame(X.loc[oos_m, :]).transpose()
        X_pred.reset_index(drop=False, inplace=True)
        X_pred.rename(columns={'index' : 'ds'}, inplace=True)
        est = est_mod.predict(X_pred)['yhat']
      elif method.endswith('AR'):
        est = est_mod.predict(ar_exog(X.loc[oos_m, X.columns], y))
      else:
        est = est_mod.predict(pd.DataFrame(X.loc[oos_m, :]).transpose())

      # Store forecast error
      fe[hor][lag][method] = np.append(fe[hor][lag][method], y[oos_m] - est)

TimeoutError: DaskDistributedBackend has no worker after 10 seconds. Make sure that workers are started and can properly connect to the scheduler and increase the joblib/dask connection timeout with:

parallel_backend('dask', wait_for_workers_timeout=20)

Using the results generated by the previous cell, the different methods can be compared. As comparison metrics, I am using the Mean Squared Prediction Error (MSPE), the Mean Absolute Prediction Error (MAPE) and their relative counterparts taking the perpetual mean as the benchmark model.

In [ ]:
'''
Construct results from the forecast errors
'''

# Define metric functions
metrics = {
  'MSPE' : lambda x: np.mean(np.power(x, 2)),
  'MAPE' : lambda x: np.mean(np.abs(x))
}

# Construct forecast metrics based on forecast errors
for (hor, lag, meth) in product(hors.keys(), lags.keys(), methods.keys()):
  for metr, metr_f in metrics.items():
    res_dict[metr + ' ' + lag][meth + ' ' + hor] = metr_f(fe[hor][lag][meth])

---
#### **Results**

In [ ]:
'''
Generate results tables
'''

# Display results table
df_res = pd.DataFrame(res_dict)
display(HTML(df_res.to_html()))

In [ ]:
'''
Generate relative results table
'''

# Display relative results table
df_rel_res = pd.concat([
  df_res.loc[
    df_res.index.to_series().apply(lambda x: x.endswith('h=1')),
    :
  ].div(df_res.loc['PM h=1', :], axis=1),
  df_res.loc[
    df_res.index.to_series().apply(lambda x: x.endswith('h=12')),
    :
  ].div(df_res.loc['PM h=12', :], axis=1)
])
display(HTML(df_rel_res.to_html()))

In [ ]:
df_res = pd.read_csv('https://raw.githubusercontent.com/Mathijs995/Applied-Macroeconometrics/main/res.csv', index_col=0)
val_cols = ['MSPE h=1', 'MAPE h=1', 'MSPE h=12', 'MAPE h=12']
df_res = df_res[val_cols]
df_res['Lag'] = df_res.index.map(lambda x: int(x.split('p=')[1]) if 'p=' in x else 1)
df_res['Method'] = df_res.index.map(lambda x: x.split()[0] if ' ' in x else x)
df_flat = pd.concat([df_res[[col, 'Lag', 'Method']] for col in val_cols])

df_flat = df_flat.loc[df_flat[val_cols].apply(lambda x: any(x.notna()), axis=1)]
df_flat['Metric'] = df_flat[val_cols].apply(lambda x: 'MSPE' if (x.notna().tolist().index(True) % 2 == 0) else 'MAPE', axis=1)
df_flat['Hor'] = df_flat[val_cols].apply(lambda x: 1 + (x.notna().tolist().index(True) // 2) * 11, axis=1)
df_flat['Value'] = df_flat[val_cols].apply(lambda x: x[x.notna()].values[0], axis=1)
df_flat.drop(columns=val_cols, inplace=True)

df_flat = pd.pivot_table(df_flat, values='Value', index=['Hor', 'Method'], columns=['Lag', 'Metric'])
df_flat.index = ['{} h={}'.format(method, hor) for hor, method in df_flat.index.values]
df_flat.at['PAR h=1', (12, 'MSPE')] = df_flat.at['PAR h=1', (1, 'MSPE')]
df_flat.at['PAR h=1', (24, 'MSPE')] = df_flat.at['PAR h=1', (1, 'MSPE')]
df_flat.at['PAR h=1', (12, 'MAPE')] = df_flat.at['PAR h=1', (1, 'MAPE')]
df_flat.at['PAR h=1', (24, 'MAPE')] = df_flat.at['PAR h=1', (1, 'MAPE')]
df_flat.at['PM h=1', (12, 'MSPE')] = df_flat.at['PM h=1', (1, 'MSPE')]
df_flat.at['PM h=1', (24, 'MSPE')] = df_flat.at['PM h=1', (1, 'MSPE')]
df_flat.at['PM h=1', (12, 'MAPE')] = df_flat.at['PM h=1', (1, 'MAPE')]
df_flat.at['PM h=1', (24, 'MAPE')] = df_flat.at['PM h=1', (1, 'MAPE')]
df_flat.at['PM h=12', (12, 'MSPE')] = df_flat.at['PM h=12', (1, 'MSPE')]
df_flat.at['PM h=12', (24, 'MSPE')] = df_flat.at['PM h=12', (1, 'MSPE')]
df_flat.at['PM h=12', (12, 'MAPE')] = df_flat.at['PM h=12', (1, 'MAPE')]
df_flat.at['PM h=12', (24, 'MAPE')] = df_flat.at['PM h=12', (1, 'MAPE')]
df_flat.columns = ['{} p={}'.format(metric, lag) for lag, metric in df_flat.columns]
val_cols = ['MSPE p=1', 'MAPE p=1', 'MSPE p=12', 'MAPE p=12', 'MSPE p=24', 'MAPE p=24']
df_flat = df_flat[val_cols]

idx = ['{} h={}'.format(method, hor) for hor in [1, 12] for method in methods.keys()]
idx.remove('PAR h=12')
df_res = df_flat.loc[idx, :]